In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from sodapy import Socrata
from app_token import app_token
from pgconnect import pgconnect

In [33]:
import json

In [11]:
import psycopg2

In [26]:
from sqlalchemy import create_engine

In [27]:
# sqlalchemy engine for using .to_sql
url = f"postgres+psycopg2://{user}:@{host}:5432/{db}"
engine = create_engine(url)

### POSTGRES CONNECTION

In [12]:
db = pgconnect['db']
host = pgconnect['host']
user = pgconnect['user']

In [14]:
con = psycopg2.connect(database=db,host=host,user=user)

In [15]:
cur = con.cursor()

In [21]:
### write the uof_filenum table
cur.execute(open('create_uof_filenum.sql').read())
con.commit() # commit the create statement to the database

In [54]:
# columns for uof_filenum table
table_cols = ['objectid',
'zip',
'filenum',
'occurred_d',
'occurred_t',
'current_ba',
'offsex',
'offrace',
'hire_dt',
'off_injure',
'offcondtyp',
'off_hospit',
'service_ty',
'uof_reason',
'cycles_num',
'street_n',
'street',
'street_g',
'street_t',
'address',
'citnum',
'citrace',
'citsex',
'cit_injure',
'citcondtyp',
'cit_arrest',
'cit_infl_a',
'citcharget',
'council_district',
'ra',
'beat',
'sector',
'division',
'x',
'y',
'geocoded_column',
'year_reported']

### API 

In [6]:
limit = 6000

In [49]:
client = Socrata("www.dallasopendata.com", app_token=app_token['token'])

In [50]:
url_dict = dict (
    data_2019 ='46zb-7qgj',
    data_2018 ='33un-ry4j',
    data_2017 ='tsu5-ca6k',
    data_2016 ='99fn-pvaf',
    data_2015 ='594v-2cnd',
    data_2014 ='xiv3-e8g7',
    data_2013 ='6gnu-avpf'
    )

In [71]:
df_2019 = pd.DataFrame(client.get('46zb-7qgj', limit=limit))

In [77]:
df_2019.head()

,objectid,zip,filenum,uofnum,occurred_d,occurred_t,current_ba,offsex,offrace,hire_dt,...,ra,beat,sector,division,x,y,geocoded_column,:@computed_region_sjyw_rtbm,:@computed_region_2f7u_b5gs,year_reported
0,2817,75253,UF2019-1702,"62295, 63542",2019-12-01T00:00:00.000,10:34 PM,11285,Male,White,2017-03-08T00:00:00.000,...,6062,357,350,SOUTHEAST,2557123.437,6944231.397,"{""type"": ""Point"", ""coordinates"": [-96.586265, ...",8,3,2019
1,2234,75208,UF2019-1344,61093,2019-10-06T00:00:00.000,12:50 AM,11208,Male,White,2016-08-24T00:00:00.000,...,4160,444,440,SOUTHWEST,2474936.793,6952151.398,"{""type"": ""Point"", ""coordinates"": [-96.853036, ...",1,3,2019
2,2755,75231,UF2019-1665,62820,2019-12-31T00:00:00.000,11:37 PM,9415,Male,White,2008-04-02T00:00:00.000,...,6034,247,240,NORTHEAST,2508349.267,7001784.466,"{""type"": ""Point"", ""coordinates"": [-96.741661, ...",13,3,2019
3,2110,75228,UF2019-1314,60990,2019-09-30T00:00:00.000,6:20 PM,9884,Male,Hispanic,2009-06-10T00:00:00.000,...,1132,228,220,NORTHEAST,2536678.324,6999039.025,"{""type"": ""Point"", ""coordinates"": [-96.649175, ...",13,3,2019
4,1663,75051,UF2019-1030,"59592, 59600",2019-08-04T00:00:00.000,12:10 AM,10480,Male,Hispanic,2012-09-26T00:00:00.000,...,None,None,None,None,2433285.622,6953645.72,"{""type"": ""Point"", ""coordinates"": [-96.98722, 3...",None,None,2019


In [72]:
# adding the year reported values
df_2019['year_reported'] = '2019'

In [73]:
# convert the geocode column from dict to JSON string for load into database
df_2019['geocoded_column'] = df_2019['geocoded_column'].apply(lambda x:json.dumps(x))

In [74]:
# replace NULL string values with None type for proper load into database
df_2019.replace(to_replace='NULL',inplace=True)

objectid                       None
zip                            None
filenum                        None
uofnum                         None
occurred_d                     None
occurred_t                     None
current_ba                     None
offsex                         None
offrace                        None
hire_dt                        None
off_injure                     None
offcondtyp                     None
off_hospit                     None
service_ty                     None
forcetype                      None
uof_reason                     None
cycles_num                     None
forceeffec                     None
street_n                       None
street                         None
street_g                       None
street_t                       None
address                        None
citnum                         None
citrace                        None
citsex                         None
cit_injure                     None
citcondtyp                  

In [75]:
# replace NaN string values with None type for proper load into database
df_2019.replace(to_replace='NaN',inplace=True)

objectid                       None
zip                            None
filenum                        None
uofnum                         None
occurred_d                     None
occurred_t                     None
current_ba                     None
offsex                         None
offrace                        None
hire_dt                        None
off_injure                     None
offcondtyp                     None
off_hospit                     None
service_ty                     None
forcetype                      None
uof_reason                     None
cycles_num                     None
forceeffec                     None
street_n                       None
street                         None
street_g                       None
street_t                       None
address                        None
citnum                         None
citrace                        None
citsex                         None
cit_injure                     None
citcondtyp                  

In [76]:
# cast all empty values into None type
df_2019.where(pd.notnull(df_2019),None,inplace=True)

In [79]:
# load into table
df_2019[table_cols].to_sql('uof_filenum',engine,schema='cdep',if_exists='append',index=False,method='multi')

## Load 2018 data

In [81]:
df_2018 = pd.DataFrame(client.get(url_dict['data_2018'], limit=limit))

In [84]:
df_2018.columns

Index(['filenum', 'uofnum', 'occurred_d', 'current_ba', 'offsex', 'offrace',
       'hire_dt', 'off_injure', 'offcondtyp', 'off_hospit', 'service_ty',
       'forcetype', 'uof_reason', 'cycles_num', 'forceeffec', 'street_n',
       'street', 'street_g', 'street_t', 'address', 'citnum', 'citrace',
       'citsex', 'cit_injure', 'citcondtyp', 'cit_arrest', 'cit_infl_a',
       'citcharget', 'ra', 'beat', 'sector', 'division', 'geocoded_column',
       ':@computed_region_sjyw_rtbm', ':@computed_region_2f7u_b5gs'],
      dtype='object')

In [87]:
# adding the year reported values
df_2018['year_reported'] = '2018'

In [88]:
# convert the geocode column from dict to JSON string for load into database
df_2018['geocoded_column'] = df_2018['geocoded_column'].apply(lambda x:json.dumps(x))

In [89]:
# replace NULL string values with None type for proper load into database
df_2018.replace(to_replace='NULL',inplace=True)

filenum                        None
uofnum                         None
occurred_d                     None
current_ba                     None
offsex                         None
offrace                        None
hire_dt                        None
off_injure                     None
offcondtyp                     None
off_hospit                     None
service_ty                     None
forcetype                      None
uof_reason                     None
cycles_num                     None
forceeffec                     None
street_n                       None
street                         None
street_g                       None
street_t                       None
address                        None
citnum                         None
citrace                        None
citsex                         None
cit_injure                     None
citcondtyp                     None
cit_arrest                     None
cit_infl_a                     None
citcharget                  

In [90]:
# replace NaN string values with None type for proper load into database
df_2018.replace(to_replace='NaN',inplace=True)

filenum                        None
uofnum                         None
occurred_d                     None
current_ba                     None
offsex                         None
offrace                        None
hire_dt                        None
off_injure                     None
offcondtyp                     None
off_hospit                     None
service_ty                     None
forcetype                      None
uof_reason                     None
cycles_num                     None
forceeffec                     None
street_n                       None
street                         None
street_g                       None
street_t                       None
address                        None
citnum                         None
citrace                        None
citsex                         None
cit_injure                     None
citcondtyp                     None
cit_arrest                     None
cit_infl_a                     None
citcharget                  

In [91]:
# cast all empty values into None type
df_2018.where(pd.notnull(df_2018),None,inplace=True)

In [101]:
# load into table
df_2018[[item for item in table_cols if item in df_2018.columns]].to_sql('uof_filenum',engine,schema='cdep',if_exists='append',index=True,index_label='objectid',method='multi')

- - -

## Experimental code for 2019 above

In [65]:
con.rollback() # for rolling back a failed transaction

In [78]:
df_2019[df_2019['geocoded_column']=='NaN']

,objectid,zip,filenum,uofnum,occurred_d,occurred_t,current_ba,offsex,offrace,hire_dt,...,ra,beat,sector,division,x,y,geocoded_column,:@computed_region_sjyw_rtbm,:@computed_region_2f7u_b5gs,year_reported


In [34]:
json.dumps(df_2019['geocoded_column'][0])

'{"type": "Point", "coordinates": [-96.586265, 32.702825]}'

In [10]:
df_2019[['off_injure',
       'offcondtyp', 'off_hospit', 'service_ty', 'forcetype', 'uof_reason',
       'cycles_num', 'forceeffec', 'street_n', 'street', 'street_g',
       'street_t', 'address', 'citnum', 'citrace', 'citsex', 'cit_injure']]

,off_injure,offcondtyp,off_hospit,service_ty,forcetype,uof_reason,cycles_num,forceeffec,street_n,street,street_g,street_t,address,citnum,citrace,citsex,cit_injure
0,false,No injuries noted or visible,false,Service Call,"BD - Tripped, BD - Grabbed",Detention/Frisk,NULL,"Yes, Yes",102,Beltline,S,Rd.,102 S Beltline Rd.,60833,White,Male,false
1,true,No injuries noted or visible,false,Arrest,Held Suspect Down,Arrest,NULL,Yes,1500,Oak Cliff,S,Blvd.,1500 S Oak Cliff Blvd.,6020748798,Hispanic,Female,true
2,false,No injuries noted or visible,false,Arrest,K-9 Deployment,Arrest,NULL,Yes,6904,Walling,NULL,Ln.,6904 Walling Ln.,61130,Black,Male,true
3,false,No injuries noted or visible,false,Call for Cover,Joint Locks,Arrest,NULL,Yes,11760,Ferguson,NULL,Rd.,11760 Ferguson Rd.,26625,White,Female,false
4,true,No injuries noted or visible,false,Arrest,"Joint Locks, BD - Grabbed",Arrest,NULL,"Yes, Yes",1350,Skyline,NULL,Rd.,1350 Skyline Rd.,59513,Black,Male,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2939,false,No injuries noted or visible,false,Suspicious Activity,Taser Display at Person,Weapon Display,NULL,Yes,1500,Park Row,NULL,Ave.,1500 Park Row Ave.,59597,Black,Male,false
2940,false,No injuries noted or visible,false,Traffic Stop,"BD - Grabbed, Verbal Command",Arrest,NULL,"Yes, Yes",3300,Illinois,W,Ave.,3300 W Illinois Ave.,58359,Black,Male,false
2941,false,No injuries noted or visible,false,Crowd Control,"Verbal Command, OC Spray, Foot Pursuit",Arrest,NULL,"No, Yes, No",3012,HENDERSON,N,Ave.,3012 N HENDERSON Ave.,54155,White,Male,true
2942,false,No injuries noted or visible,false,Crime in Progress,"Verbal Command, Foot Pursuit",Weapon Display,NULL,"No, No",11217,Garland,NULL,Rd.,11217 Garland Rd.,58106,Asian,Male,false
